# Feature Engineering

---

1. Import packages
2. Load data
3. Feature engineering

---

## 1. Import packages

In [1]:
import pandas as pd
import pickle
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from skimpy import skim
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay,classification_report

---
## 2. Load data

In [2]:
df = pd.read_csv('D:\Projects\BCG_Internship\src\data\clean_data_after_eda.csv')
df["date_activ"] = pd.to_datetime(df["date_activ"], format='%Y-%m-%d')
df["date_end"] = pd.to_datetime(df["date_end"], format='%Y-%m-%d')
df["date_modif_prod"] = pd.to_datetime(df["date_modif_prod"], format='%Y-%m-%d')
df["date_renewal"] = pd.to_datetime(df["date_renewal"], format='%Y-%m-%d')

In [3]:
df.head()

,id,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,...,mean_3m_price_p1_var,mean_3m_price_p2_var,mean_3m_price_p3_var,mean_3m_price_p1_fix,mean_3m_price_p2_fix,mean_3m_price_p3_fix,mean_3m_price_p1,mean_3m_price_p2,mean_3m_price_p3,churn
0,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.00,...,0.131756,0.092638,0.036909,42.497907,12.218665,8.145777,42.629663,12.311304,8.182687,1
1,d29c2c54acc38ff3c0614d0a653813dd,MISSING,4660,0,0,2009-08-21,2016-08-30,2009-08-21,2015-08-31,189.95,...,0.147600,0.000000,0.000000,44.444710,0.000000,0.000000,44.592310,0.000000,0.000000,0
2,764c75f661154dac3a6c254cd082ea7d,foosdfpfkusacimwkcsosbicdxkicaua,544,0,0,2010-04-16,2016-04-16,2010-04-16,2015-04-17,47.96,...,0.167798,0.088409,0.000000,44.444710,0.000000,0.000000,44.612508,0.088409,0.000000,0
3,bba03439a292a1e166f80264c16191cb,lmkebamcaaclubfxadlmueccxoimlema,1584,0,0,2010-03-30,2016-03-30,2010-03-30,2015-03-31,240.04,...,0.148586,0.000000,0.000000,44.444710,0.000000,0.000000,44.593296,0.000000,0.000000,0
4,149d57cf92fc41cf94415803a877cb4b,MISSING,4425,0,526,2010-01-13,2016-03-07,2010-01-13,2015-03-09,445.75,...,0.119906,0.101673,0.073719,40.728885,24.437330,16.291555,40.848791,24.539003,16.365274,0


---

## 3. Feature engineering

### Difference between off-peak prices in December and preceding January

Below is the code created by your colleague to calculate the feature described above.
Use this code to re-create this feature and then think about ways to build on this feature
to create features with a higher predictive power.

In [4]:
price_df = pd.read_csv('D:\Projects\BCG_Internship\src\Data\price_data.csv')
price_df["price_date"] = pd.to_datetime(price_df["price_date"], format='%Y-%m-%d')
price_df.head()

,id,price_date,price_off_peak_var,price_peak_var,price_mid_peak_var,price_off_peak_fix,price_peak_fix,price_mid_peak_fix
0,038af19179925da21a25619c5a24b745,2015-01-01,0.151367,0.0,0.0,44.266931,0.0,0.0
1,038af19179925da21a25619c5a24b745,2015-02-01,0.151367,0.0,0.0,44.266931,0.0,0.0
2,038af19179925da21a25619c5a24b745,2015-03-01,0.151367,0.0,0.0,44.266931,0.0,0.0
3,038af19179925da21a25619c5a24b745,2015-04-01,0.149626,0.0,0.0,44.266931,0.0,0.0
4,038af19179925da21a25619c5a24b745,2015-05-01,0.149626,0.0,0.0,44.266931,0.0,0.0


In [5]:
# Group off-peak prices by companies and month
monthly_price_by_id = price_df.groupby(['id', 'price_date']).agg({'price_off_peak_var': 'mean', 'price_off_peak_fix': 'mean'}).reset_index()

# Get january and december prices
jan_prices = monthly_price_by_id.groupby('id').first().reset_index()
dec_prices = monthly_price_by_id.groupby('id').last().reset_index()

# Calculate the difference
diff = pd.merge(dec_prices.rename(columns={'price_off_peak_var': 'dec_1', 'price_off_peak_fix': 'dec_2'}), jan_prices.drop(columns='price_date'), on='id')
diff['offpeak_diff_dec_january_energy'] = diff['dec_1'] - diff['price_off_peak_var']
diff['offpeak_diff_dec_january_power'] = diff['dec_2'] - diff['price_off_peak_fix']
diff = diff[['id', 'offpeak_diff_dec_january_energy','offpeak_diff_dec_january_power']]
diff.head()

,id,offpeak_diff_dec_january_energy,offpeak_diff_dec_january_power
0,0002203ffbb812588b632b9e628cc38d,-0.006192,0.162916
1,0004351ebdd665e6ee664792efc4fd13,-0.004104,0.177779
2,0010bcc39e42b3c2131ed2ce55246e3c,0.050443,1.500000
3,0010ee3855fdea87602a5b7aba8e42de,-0.010018,0.162916
4,00114d74e963e47177db89bc70108537,-0.003994,-0.000001


In [6]:
df1 = pd.merge(df, diff, on='id').reset_index(drop=True)
df1.head(3)

,id,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,...,mean_3m_price_p3_var,mean_3m_price_p1_fix,mean_3m_price_p2_fix,mean_3m_price_p3_fix,mean_3m_price_p1,mean_3m_price_p2,mean_3m_price_p3,churn,offpeak_diff_dec_january_energy,offpeak_diff_dec_january_power
0,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.00,...,0.036909,42.497907,12.218665,8.145777,42.629663,12.311304,8.182687,1,0.020057,3.700961
1,d29c2c54acc38ff3c0614d0a653813dd,MISSING,4660,0,0,2009-08-21,2016-08-30,2009-08-21,2015-08-31,189.95,...,0.000000,44.444710,0.000000,0.000000,44.592310,0.000000,0.000000,0,-0.003767,0.177779
2,764c75f661154dac3a6c254cd082ea7d,foosdfpfkusacimwkcsosbicdxkicaua,544,0,0,2010-04-16,2016-04-16,2010-04-16,2015-04-17,47.96,...,0.000000,44.444710,0.000000,0.000000,44.612508,0.088409,0.000000,0,-0.004670,0.177779


In [7]:
#creating new feature contract period in days using date_active and date_end features
df1['contract_period_in_days']=(df1['date_end']-df1['date_activ']).dt.days

# Removing unwanted columns

*  droping columns which are useless and used in feature engineering to create new meaningfull features

In [8]:
df1.drop(columns=['price_date_x','mean_year_price_p1_var', 'mean_year_price_p2_var',
       'mean_year_price_p3_var', 'mean_year_price_p1_fix',
       'mean_year_price_p2_fix', 'mean_year_price_p3_fix','price_date_y', 'mean_6m_price_p1_var', 'mean_6m_price_p2_var',
       'mean_6m_price_p3_var', 'mean_6m_price_p1_fix', 'mean_6m_price_p2_fix',
       'mean_6m_price_p3_fix','price_date', 'mean_3m_price_p1_var',
       'mean_3m_price_p2_var', 'mean_3m_price_p3_var', 'mean_3m_price_p1_fix',
       'mean_3m_price_p2_fix', 'mean_3m_price_p3_fix','date_modif_prod', 'date_renewal','date_activ', 'date_end'],inplace=True)

#to reduce complexity of model
df1.drop(columns=['channel_sales'],inplace=True)

#we have feature net_margin so below features will providing some or less same information 
df1.drop(columns=['margin_gross_pow_ele', 'margin_net_pow_ele'],inplace=True)



In [9]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14605 entries, 0 to 14604
Data columns (total 31 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   id                               14605 non-null  object 
 1   cons_12m                         14605 non-null  int64  
 2   cons_gas_12m                     14605 non-null  int64  
 3   cons_last_month                  14605 non-null  int64  
 4   forecast_cons_12m                14605 non-null  float64
 5   forecast_cons_year               14605 non-null  int64  
 6   forecast_discount_energy         14605 non-null  float64
 7   forecast_meter_rent_12m          14605 non-null  float64
 8   forecast_price_energy_off_peak   14605 non-null  float64
 9   forecast_price_energy_peak       14605 non-null  float64
 10  forecast_price_pow_off_peak      14605 non-null  float64
 11  has_gas                          14605 non-null  object 
 12  imp_cons          

# Converting categorical data to numerical data

In [10]:
cat_features=[f for f in df1.columns if df1[f].dtype == 'object']
cat_features

['id', 'has_gas', 'origin_up']

In [11]:
#using bin encoding because there are only two unique categorical values 
df1['has_gas']=df1['has_gas'].map({'t':1,'f':0})

In [12]:
#using one hot encoding to convert "origin_up" to numerical columns
# Ensure that 'origin_up' column only contains categorical data


In [13]:
df1=pd.get_dummies(df1,columns=['origin_up'],dtype='int')
df1.head()

,id,cons_12m,cons_gas_12m,cons_last_month,forecast_cons_12m,forecast_cons_year,forecast_discount_energy,forecast_meter_rent_12m,forecast_price_energy_off_peak,forecast_price_energy_peak,...,churn,offpeak_diff_dec_january_energy,offpeak_diff_dec_january_power,contract_period_in_days,origin_up_MISSING,origin_up_ewxeelcelemmiwuafmddpobolfuxioce,origin_up_kamkkxfxxuwbdslkwifmmcsiusiuosws,origin_up_ldkssxwpmemidmecebumciepifcamkci,origin_up_lxidpiddsbxsbosboudacockeimpuepw,origin_up_usapbepcfoloekilkwsdiboslwaxobdp
0,24011ae4ebbe3035111d65fa7c15bc57,0,54946,0,0.00,0,0.0,1.78,0.114481,0.098142,...,1,0.020057,3.700961,1096,0,0,0,0,1,0
1,d29c2c54acc38ff3c0614d0a653813dd,4660,0,0,189.95,0,0.0,16.27,0.145711,0.000000,...,0,-0.003767,0.177779,2566,0,0,1,0,0,0
2,764c75f661154dac3a6c254cd082ea7d,544,0,0,47.96,0,0.0,38.72,0.165794,0.087899,...,0,-0.004670,0.177779,2192,0,0,1,0,0,0
3,bba03439a292a1e166f80264c16191cb,1584,0,0,240.04,0,0.0,19.83,0.146694,0.000000,...,0,-0.004547,0.177779,2192,0,0,1,0,0,0
4,149d57cf92fc41cf94415803a877cb4b,4425,0,526,445.75,526,0.0,131.73,0.116900,0.100015,...,0,-0.006192,0.162916,2245,0,0,1,0,0,0


In [14]:
train_data=df1.drop(columns='id')
train_data.head(3)

,cons_12m,cons_gas_12m,cons_last_month,forecast_cons_12m,forecast_cons_year,forecast_discount_energy,forecast_meter_rent_12m,forecast_price_energy_off_peak,forecast_price_energy_peak,forecast_price_pow_off_peak,...,churn,offpeak_diff_dec_january_energy,offpeak_diff_dec_january_power,contract_period_in_days,origin_up_MISSING,origin_up_ewxeelcelemmiwuafmddpobolfuxioce,origin_up_kamkkxfxxuwbdslkwifmmcsiusiuosws,origin_up_ldkssxwpmemidmecebumciepifcamkci,origin_up_lxidpiddsbxsbosboudacockeimpuepw,origin_up_usapbepcfoloekilkwsdiboslwaxobdp
0,0,54946,0,0.00,0,0.0,1.78,0.114481,0.098142,40.606701,...,1,0.020057,3.700961,1096,0,0,0,0,1,0
1,4660,0,0,189.95,0,0.0,16.27,0.145711,0.000000,44.311378,...,0,-0.003767,0.177779,2566,0,0,1,0,0,0
2,544,0,0,47.96,0,0.0,38.72,0.165794,0.087899,44.311378,...,0,-0.004670,0.177779,2192,0,0,1,0,0,0


# outlier removal and scaling

# from previous EDA session, we can say  that that is not in standard from and data is skewed ,but we are going use Random Forest classifier
* 1) which is less sensitive to outliers and
* 2) scaling does not require,because it is tree based technique ,but it may give little more predictive power

In [15]:
skewed = [
    'cons_12m', 
    'cons_gas_12m', 
    'cons_last_month',
    'forecast_cons_12m', 
    'forecast_cons_year', 
    'forecast_discount_energy',
    'forecast_meter_rent_12m', 
    'forecast_price_energy_off_peak',
    'forecast_price_energy_peak', 
    'forecast_price_pow_off_peak'
]

train_data[skewed].describe()

,cons_12m,cons_gas_12m,cons_last_month,forecast_cons_12m,forecast_cons_year,forecast_discount_energy,forecast_meter_rent_12m,forecast_price_energy_off_peak,forecast_price_energy_peak,forecast_price_pow_off_peak
count,1.460500e+04,1.460500e+04,14605.000000,14605.000000,14605.000000,14605.000000,14605.000000,14605.000000,14605.000000,14605.000000
mean,1.592303e+05,2.809108e+04,16091.371448,1868.638618,1399.858747,0.966450,63.090448,0.137282,0.050488,43.130085
std,5.734836e+05,1.629786e+05,64366.262314,2387.651549,3247.876793,5.108355,66.166636,0.024623,0.049037,4.486140
min,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,5.674000e+03,0.000000e+00,0.000000,494.980000,0.000000,0.000000,16.180000,0.116340,0.000000,40.606701
50%,1.411600e+04,0.000000e+00,793.000000,1112.610000,314.000000,0.000000,18.800000,0.143166,0.084138,44.311378
75%,4.076400e+04,0.000000e+00,3383.000000,2402.270000,1746.000000,0.000000,131.030000,0.146348,0.098837,44.311378
max,6.207104e+06,4.154590e+06,771203.000000,82902.830000,175375.000000,30.000000,599.310000,0.273963,0.195975,59.266378


In [16]:
# Apply log10 transformation
train_data["cons_12m"] = np.log10(train_data["cons_12m"] + 1)
train_data["cons_gas_12m"] = np.log10(train_data["cons_gas_12m"] + 1)
train_data["cons_last_month"] = np.log10(train_data["cons_last_month"] + 1)
train_data["forecast_cons_12m"] = np.log10(train_data["forecast_cons_12m"] + 1)
train_data["forecast_cons_year"] = np.log10(train_data["forecast_cons_year"] + 1)
train_data["forecast_meter_rent_12m"] = np.log10(train_data["forecast_meter_rent_12m"] + 1)
train_data["imp_cons"] = np.log10(df["imp_cons"] + 1)

In [17]:
train_data[skewed].describe()

,cons_12m,cons_gas_12m,cons_last_month,forecast_cons_12m,forecast_cons_year,forecast_discount_energy,forecast_meter_rent_12m,forecast_price_energy_off_peak,forecast_price_energy_peak,forecast_price_pow_off_peak
count,14605.000000,14605.000000,14605.000000,14605.000000,14605.000000,14605.000000,14605.000000,14605.000000,14605.000000,14605.000000
mean,4.223945,0.778978,2.264801,2.962162,1.784733,0.966450,1.517233,0.137282,0.050488,43.130085
std,0.884545,1.716828,1.769266,0.683612,1.584972,5.108355,0.571489,0.024623,0.049037,4.486140
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.753966,0.000000,0.000000,2.695464,0.000000,0.000000,1.235023,0.116340,0.000000,40.606701
50%,4.149742,0.000000,2.899821,3.046733,2.498311,0.000000,1.296665,0.143166,0.084138,44.311378
75%,4.610287,0.000000,3.529430,3.380803,3.242293,0.000000,2.120673,0.146348,0.098837,44.311378
max,6.792889,6.618528,5.887169,4.918575,5.243970,30.000000,2.778376,0.273963,0.195975,59.266378


# Model building

In [18]:
X=train_data.drop(columns=['churn'])
y=train_data['churn']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [19]:
X_train.shape, y_train.shape

((11684, 34), (11684,))

In [20]:
RD=RandomForestClassifier()
RD.fit(X_train,y_train)
print("f1_score:",f1_score(y_test,RD.predict(X_test),average='macro'))

f1_score: 0.5391675925826043


In [21]:
# feature importance slection
importances = RD.feature_importances_

features = pd.DataFrame({'feature':X.columns, 'importance': importances})
features.sort_values(by='importance', ascending=False, inplace=True)

print(features)

                                       feature    importance
0                                     cons_12m  7.936514e-02
6                      forecast_meter_rent_12m  7.812946e-02
13                                  net_margin  7.806813e-02
3                            forecast_cons_12m  7.362385e-02
16                          mean_year_price_p1  5.709300e-02
15                                     pow_max  5.308145e-02
2                              cons_last_month  5.193874e-02
25             offpeak_diff_dec_january_energy  4.869420e-02
19                            mean_6m_price_p1  4.523194e-02
27                     contract_period_in_days  4.388524e-02
11                                    imp_cons  4.222646e-02
4                           forecast_cons_year  4.130640e-02
7               forecast_price_energy_off_peak  3.833619e-02
22                            mean_3m_price_p1  3.660999e-02
17                          mean_year_price_p2  2.926072e-02
20                      

In [22]:
# Set a threshold
threshold = 0.01

# Find features that meet the threshold
important_features = features[features['importance'] > threshold]['feature']

# Create a new DataFrame with only important features
X_imp_feat = X[important_features]


In [23]:
X1=X_imp_feat
y1=train_data['churn']
X1_train,X1_test,y1_train,y1_test=train_test_split(X1,y1,test_size=0.2,random_state=42)

RD=RandomForestClassifier()
RD.fit(X1_train,y1_train)
print("f1_score:",f1_score(y1_test,RD.predict(X1_test),average='macro'))

f1_score: 0.5213457613086486


In [26]:
X1.shape,y1.shape

((14605, 25), (14605,))

In [27]:
X1.to_csv(r'D:\Projects\BCG_Internship\Artifacts\train.csv',index=False)
y1.to_csv(r'D:\Projects\BCG_Internship\Artifacts\test.csv',index=False)

In [63]:
import pickle

with open(r'D:\Projects\BCG_Internship\Artifacts\classifier.pkl', 'wb') as obj:
    pickle.dump(RD, obj)


In [53]:
# Aare under curve
y_pred = RD.predict_proba(X1_test)[:, 1]

# Calculate AUC
auc = roc_auc_score(y1_test, y_pred)
print('AUC: %.3f' % auc)

AUC: 0.615


# Conclusion

1) I have used RandomForestClassifier() for following reasons
* less sensitive to outlier
* scaling is not required
* a robust algorithm
2) I choose evaluation metrics
* AUC - because -> The AUC (Area Under the Curve) represents the degree or measure of separability. It   tells how much the model is capable of distinguishing between classes. Higher the AUC, better the     model is at predicting 0s as 0s and 1s as 1s
* F1-score -> It combines precision and recall and usefull for imbalanced class

3) Advantage of using RandomForestClassifier()  -> In this case is that, data is highly skewed and contains too many outliers
   Disadvantage of using RandomForestClassifier()->
   a) Less Interpretability: because it involves multiple trees
   b) Training Time and Memory Usage: Random Forest can be computationally expensive. It requires more       time to train and more memory to store the multiple decision trees.
   c) slower than other algorithms
4) model scored 0.655 AUC score ,which is a good score with such a noisy data

5) * by parameter_importance we can say that, price sensitivity also a reason for churn, but not major player   

In [25]:
from sklearn import metrics
print(f"Accuracy: {metrics.accuracy_score(y1_test,RD.predict(X1_test))}")
print(f"Precision: {metrics.precision_score(y1_test,RD.predict(X1_test))}")
print(f"Recall: {metrics.recall_score(y1_test,RD.predict(X1_test))}")

Accuracy: 0.9020883259157823
Precision: 0.9285714285714286
Recall: 0.0436241610738255


# Business impact of discount
We conduct a brief analysis on the discount strategy proposed. This is just 1 way to solve it, there may be many other ways!

The SME division head proposed that we give a 20% discount to high propensity to churn customers. We can assume to start that everyone who is offered a discount will accept it.

Workflow
Our task is to calculate the forecast revenue for the set of customers:

1) When no discount is offered
2) And when a discount is offered based on a probability cutoff to decide who should receive the 20% discount
And hence to decide where the cut-off should be set in order to maximise revenue

In [27]:
test_df = train_data
test_df.head()

,cons_12m,cons_gas_12m,cons_last_month,forecast_cons_12m,forecast_cons_year,forecast_discount_energy,forecast_meter_rent_12m,forecast_price_energy_off_peak,forecast_price_energy_peak,forecast_price_pow_off_peak,...,churn,offpeak_diff_dec_january_energy,offpeak_diff_dec_january_power,contract_period_in_days,origin_up_MISSING,origin_up_ewxeelcelemmiwuafmddpobolfuxioce,origin_up_kamkkxfxxuwbdslkwifmmcsiusiuosws,origin_up_ldkssxwpmemidmecebumciepifcamkci,origin_up_lxidpiddsbxsbosboudacockeimpuepw,origin_up_usapbepcfoloekilkwsdiboslwaxobdp
0,0.000000,4.739944,0.000000,0.000000,0.000000,0.0,0.444045,0.114481,0.098142,40.606701,...,1,0.020057,3.700961,1096,0,0,0,0,1,0
1,3.668479,0.000000,0.000000,2.280920,0.000000,0.0,1.237292,0.145711,0.000000,44.311378,...,0,-0.003767,0.177779,2566,0,0,1,0,0,0
2,2.736397,0.000000,0.000000,1.689841,0.000000,0.0,1.599009,0.165794,0.087899,44.311378,...,0,-0.004670,0.177779,2192,0,0,1,0,0,0
3,3.200029,0.000000,0.000000,2.382089,0.000000,0.0,1.318689,0.146694,0.000000,44.311378,...,0,-0.004547,0.177779,2192,0,0,1,0,0,0
4,3.646011,0.000000,2.721811,2.650065,2.721811,0.0,2.122969,0.116900,0.100015,40.606701,...,0,-0.006192,0.162916,2245,0,0,1,0,0,0


* Calculate a baseline revenue estimate (no intervention)
Calculate a baseline estimate of the electricity revenue for every customer for the next twelve months based on the forecast consumption and forecast price and actual churn outcome. Call this basecase_revenue.

* For customers who end up churning, we should reduce our forecast revenue calculation by 91.9% to account for the customers churn some time between January 2016 and the start of March 2016. (Not knowing when they churn, a reasonable assumption for the lost revenue is the average of 100%, corresponding to churn on 1 January 2016, and 83.9%, corresponding to churn at the end of February, or 59 days into a 365 day year). Call this new variable basecase_revenue_after_churn, ie basecase_revenue_after_churn = basecase_revenue*(1 -0.919 * churn)

In [28]:
# Electricity revenue for each customer consists of energy consumption (amount * price) and the meter rent
# (the power price may also play a role, but we will ignore it for now since we need to ask the client for more data)
# Note that we need to reverse the log10-transformation from the data cleaning step
test_df['basecase_revenue'] = (np.power(10, test_df['forecast_cons_12m']) + 1) * test_df['forecast_price_energy_off_peak'] + test_df['forecast_meter_rent_12m']

# Taking churn into account
test_df['basecase_revenue_after_churn'] = test_df['basecase_revenue'] * (1 - 0.919 * test_df['churn'])

In [29]:
test_df.head()

,cons_12m,cons_gas_12m,cons_last_month,forecast_cons_12m,forecast_cons_year,forecast_discount_energy,forecast_meter_rent_12m,forecast_price_energy_off_peak,forecast_price_energy_peak,forecast_price_pow_off_peak,...,offpeak_diff_dec_january_power,contract_period_in_days,origin_up_MISSING,origin_up_ewxeelcelemmiwuafmddpobolfuxioce,origin_up_kamkkxfxxuwbdslkwifmmcsiusiuosws,origin_up_ldkssxwpmemidmecebumciepifcamkci,origin_up_lxidpiddsbxsbosboudacockeimpuepw,origin_up_usapbepcfoloekilkwsdiboslwaxobdp,basecase_revenue,basecase_revenue_after_churn
0,0.000000,4.739944,0.000000,0.000000,0.000000,0.0,0.444045,0.114481,0.098142,40.606701,...,3.700961,1096,0,0,0,0,1,0,0.673007,0.054514
1,3.668479,0.000000,0.000000,2.280920,0.000000,0.0,1.237292,0.145711,0.000000,44.311378,...,0.177779,2566,0,0,1,0,0,0,29.206519,29.206519
2,2.736397,0.000000,0.000000,1.689841,0.000000,0.0,1.599009,0.165794,0.087899,44.311378,...,0.177779,2192,0,0,1,0,0,0,9.882077,9.882077
3,3.200029,0.000000,0.000000,2.382089,0.000000,0.0,1.318689,0.146694,0.000000,44.311378,...,0.177779,2192,0,0,1,0,0,0,36.824505,36.824505
4,3.646011,0.000000,2.721811,2.650065,2.721811,0.0,2.122969,0.116900,0.100015,40.606701,...,0.162916,2245,0,0,1,0,0,0,54.464944,54.464944


Calculate the estimated benefits and costs of intervention
Now, pick a cut-off probability (eg 0.5) so that:

* Customers with a higher churn probability than the cut-off get a discount, and
* Customers below the churn-probability do not get a discount.
From this, calculate the revenue of the intervention scenario assuming:

* All customers who are offered a discount accept it
* Customers who do receive a discount are are assumed not to churn in the next twelve months (ie churn probability = 0), and therefore the retained revenue is 0.8*basecase_revenue , being (1-discount_fraction)*basecase_revenue
* Customers who do not receive a discount are assumed to churn based on the observed dependent variable (ie, a 1 or 0 for whether they actually churned or not).

Now, map out the revenue delta as a function of the cut-off probability in a graph. What cut-off probability approximately optimises the revenue outcome? Assume for these calculations that the customer does not consume more or less electricity because the price changes. In practice, we would expect that if the customer's cost goes down then their consumption might increase. We will see two counterbalancing effects at play:

* For true positives we will see revenue retention vs the no-discount scenario
* For false positives we will see reduced revenue from giving them a discount when they wouldn't in fact churn.

(False negatives represent an opportunity cost but not an actual cost difference between the two scenarios.)

The optimal cut-off point will balance the benefits from true positives against the costs of false positives. Our task is to approximately find the optimal cut-off point. We may need to make additional assumptions. If we feel the assumptions above aren't justified and that others are better then we should modify our assumptions.